In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import fiona

In [2]:
cnty = {'Alameda': 1.0,
'Contra Costa': 13.0,
'Marin': 41.0,
'Napa': 55.0,
'San Francisco': 75.0,
'San Mateo': 81.0,
'Santa Clara': 85.0,
'Solano': 95.0,
'Sonoma': 97.0}

ctyMap = pd.DataFrame(cnty.items(), columns=['ctyName', 'ctyCode'])

## 0 Data sources

In [3]:
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\PLU_analysis\PLU_analysis.gdb'))

# Input files
## Pacel 10
p10_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\PLU_analysis\PLU_analysis.gdb', layer='p10_table')

## parcel10 to pba40 basezoning code
pz10 = pd.read_csv(r'C:\Users\ywang\Box\Mine\1_UrbanSim\2020_03_06_zoning_parcels.csv')

## pba40 basezoning plu
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')

## pba50 basezoning PLU
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\UrbanSim Data Review Option B v1d.gdb\UrbanSim Data Review Option B v1d.gdb\UrbanSim Data Review Option B v1d.gdb'))
p10_plu50_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\UrbanSim Data Review Option B v1d.gdb\UrbanSim Data Review Option B v1d.gdb\UrbanSim Data Review Option B v1d.gdb', layer='p10_boc_opt_b_v1d_geo_yq')

## planned zoning scenarios
zmods = pd.read_csv('C:/Users/ywang/Box/Mine/1_UrbanSim/03_06_2020_parcels_geography.csv')

## Building data to decide parcel status
blg10 = pd.read_csv('blg10.csv')

['p10_table', 'p10_boc_v3_geo_tbl_20200311', 'p10_pba50_tbl']
['p10_boc_opt_b_v1d_geo_yq']


In [ ]:
nonRes = ['OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']
Res = ['HS', 'HT', 'HM','MR']

## 1 Merge data sets

### 1.1 P10 parcel zoining designations

In [4]:
# parcel geometry
p10 = p10_raw[['PARCEL_ID','geom_id_s','COUNTY_ID','jurisdiction',
                           'ACRES','LAND_VALUE','pda_id','zoningmodcat']]
print(p10.shape)
#display(p10.head())

# pacel to zoning code mapping
print(pz10.shape)
#display(pz10.head())

p10_z10 = p10.merge(pz10, on = 'PARCEL_ID', how = 'left')
print(p10_z10.shape)
display(p10_z10.head())

# Check Number of parcels missing zoning designation
z10_missing = p10_z10.loc[p10_z10['nodev_pba40'].isnull()]
print(z10_missing.shape[0])
print(z10_missing.shape[0]/pz10.shape[0])

(1956208, 8)
(1950733, 11)
(1956208, 18)


,PARCEL_ID,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,nodev,juris_id
0,229116.0,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1771381.0,1.030511e+13,60126.0,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0,0.0,livr
1,244166.0,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1362648.0,1.110735e+13,11903.0,GP-ULM,99.0,100.0,livermoregeneralplan,0.0,0.0,livr
2,202378.0,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,307258.0,1.103018e+13,11803.0,LDR,98.0,100.0,hayward_gp_landuse,0.0,0.0,hayw
3,2004420.0,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,1737625.0,6.381678e+12,12975.0,LEA240,109.0,100.0,sonomacountygeneralplan,0.0,0.0,uson
4,340332.0,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,273989.0,3.148755e+11,2511.0,RESM4,5.0,100.0,fremontgeneralplan,0.0,1.0,frem


5476
0.002807149927745109


### 1.2 parcel 10 with PBA40 zoning code PLU

In [5]:
# check duplicates in zoning id
plu10['id'] = plu10['id'].apply(lambda x: float(x))
plu10['jz_o'] = plu10['city'].str.cat(plu10['name'],sep=" ")
print(plu10.shape[0], len(plu10.id.unique()), len(plu10.jz_o.unique()))

# relabel p10 land plu info (used in PBA40)
cols = [i+'_10' for i in list(plu10)]
plu10.columns = cols
#display(plu10.head())

# merge PBA40 plu to p10
p10_plu10 = p10_z10.merge(plu10, left_on = 'zoning_id', right_on = 'id_10', how = 'left')
#display(p10_plu10.head())

# Check number of p10 records failed to find a matching PLU
#display(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()])
print(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()].shape[0] / p10_z10.shape[0])

5156 5156 4536
0.0034745793903306807


### 1.3 P10 with BASIS BOC

In [60]:
plu50 = p10_plu50_raw[['parcel_id_p10','me','mt', 'mr', 'rb', 'rs', 'ih', 'iw', 'il', 'sc', 'ho', 'of', 'hm', 'ht', 'hs',
                       'max_height','max_dua','max_far','plu_id','plu_jurisdiction','plu_description','building_types_source','source']]

# relabel BASIS land plu info (to use in PBA50)
cols2 = [i+'_18' for i in list(plu50)]
plu50.columns = cols2
#display(plu50.head())

# merge PBA50 plu to p10
p10_plus = p10_plu10.merge(plu50, left_on = 'PARCEL_ID', right_on = 'parcel_id_p10_18', how = 'left')

p10_plus.drop(columns = ['zoning','tablename','Unnamed: 0','id_10','name_10','plandate_10','jz_o_10','parcel_id_p10_18'],inplace = True)
#display(p10_plus.head())

(1956208, 23)
(1956208, 65)


### 1.4 Bring in Building data (b10) to determine parcel characteristics

In [61]:
print(blg10.shape[0], len(blg10.building_id.unique()), len(blg10.parcel_id.unique()))
#display(blg10.head())

# Assign parcel characteristics

# merge builing and parcel data w/ Outer-join
b10_p10 = blg10.merge(p10[['PARCEL_ID']],left_on = 'parcel_id',right_on = 'PARCEL_ID', how = 'outer')
print(b10_p10.shape)

# sum all values for multiple buildings within one parcel
pb10_v = b10_p10.groupby(['PARCEL_ID'])['improvement_value','residential_units','residential_sqft','non_residential_sqft',
                                      'building_sqft','redfin_sale_price','costar_rent'].sum().reset_index()

# chose the earliest built year for multiple buildings within one parcel
pb10_yr = b10_p10.groupby(['PARCEL_ID'])['year_built','building_id'].min().reset_index()

# parcel vacancy based on building type
b10_p10['dType'] = b10_p10['development_type_id']
blg10.loc[blg10['development_type_id'] == 0, 'dType'] = 'Vacant'
blg10.loc[blg10['development_type_id'] == 15, 'dType'] = 'Vacant'
pb10_vacent = b10_p10.loc[b10_p10['dType'] == 'Vacant'][['PARCEL_ID','dType']]

# merge
pb10_temp = pb10_v.merge(pb10_yr, on = 'PARCEL_ID', how = 'left').merge(pb10_vacent, on = 'PARCEL_ID', how = 'left')
print(pb10_temp.shape)
pb10_plus = p10_plus.merge(pb10_temp, on = 'PARCEL_ID', how = 'left')

# Investment-land ratio
pb10_plus['ILR'] = pb10_plus['improvement_value'] / pb10_plus['LAND_VALUE']
pb10_plus.loc[pb10_plus['LAND_VALUE'] == 0, 'ILR'] = 'n/a'

# Vacant parcels
pb10_plus['vacant'] = np.where((pb10_plus['building_id'].isnull()) | (pb10_plus['dType'] == 'Vacant') | 
                          ((pb10_plus['improvement_value'] == 0) & (pb10_plus['residential_units'] == 0) & 
                             (pb10_plus['residential_sqft'] == 0) & (pb10_plus['non_residential_sqft'] == 0) &
                             (pb10_plus['building_sqft'] == 0)), 'vacant', 'nonVacant')

#display(pb10_plus.head())

1843351 1843351 1843292


,Unnamed: 0,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,...,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,id,geometry,dType
0,0,1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,...,1,1945,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,1,2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,...,1,1965,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,2,3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,...,1,1964,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3,4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,...,2,1992,340000.0,2003.0,Condo/Coop,NaN,NaN,4,NaN,NaN
4,4,5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,...,1,1978,442000.0,2004.0,Condo/Coop,NaN,NaN,5,NaN,NaN


(1956269, 23)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  from ipykernel import kernelapp as app
C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(1956208, 10)


,PARCEL_ID,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,geom_id,zoning_id,...,residential_units,residential_sqft,non_residential_sqft,building_sqft,redfin_sale_price,year_built,building_id,dType,ILR,vacant
0,229116.0,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,60126.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
1,244166.0,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,11903.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
2,202378.0,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,11803.0,...,20.0,101000.0,0.0,101000.0,1007250.0,2009.0,15681.0,NaN,0,nonVacant
3,2004420.0,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,12975.0,...,0.0,0.0,0.0,0.0,0.0,1965.0,17798.0,NaN,0.812491,nonVacant
4,340332.0,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,2511.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant


### 1.5 Bring in zoning scenarios data

In [62]:
zmods.columns = list(zmods)[:-2] + ['nodev_pba50','jurisdiction_id']
#display(zmods.head())

# merge parcel data with zoning mods
pb10_plus_zmods = pb10_plus.merge(zmods, on = 'geom_id', how = 'left')
#display(pb10_plus_zmods.head())
pb10_plus_zmods.columns = [x.upper() for x in pb10_plus_zmods.columns]

(1956208, 26)
(1956208, 93)


### 1.6 Export BOC data for mapping

In [67]:
p10_plu_boc = pb10_plus_zmods[['PARCEL_ID_X','COUNTY_ID','JURIS_ID_Y','PLU_ID_18','PLU_JURISDICTION_18','PLU_DESCRIPTION_18',
                    'MAX_FAR_10','MAX_DUA_10','MAX_DUA_18','MAX_FAR_18',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
                    'ME_18','MT_18','MR_18','RB_18','RS_18','IH_18','IW_18','IL_18','SC_18','HO_18','OF_18','HM_18','HT_18','HS_18',
                    'BUILDING_TYPES_SOURCE_18','SOURCE_18']]

In [68]:
p10_plu_boc.columns = ['PARCEL_ID','COUNTY_ID','JURIS_NEW'] + list(p10_plu_boc)[3:]
p10_plu_boc.to_csv('outputs/p10_plu_boc_all.csv',index = False)

## 2 Capacity statistics

In [128]:
# select needed fields
plu_main = pb10_plus_zmods.loc[pb10_plus_zmods['COUNTY_ID'] > 0][['PARCEL_ID_X','COUNTY_ID','JURIS_ID_Y','ZONING_ID','GEOM_ID_S','ACRES',
                    'MAX_FAR_10','MAX_DUA_10','MAX_DUA_18','MAX_FAR_18', 'MAX_HEIGHT_10','MAX_HEIGHT_18',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
                    'ME_18','MT_18','MR_18','RB_18','RS_18','IH_18','IW_18','IL_18','SC_18','HO_18','OF_18','HM_18','HT_18','HS_18',
                    'YEAR_BUILT','ILR','VACANT','PBA50ZONINGMODCAT','NODEV_PBA40','NODEV_PBA50',
                    'BUILDING_TYPES_SOURCE_18','SOURCE_18']]
plu_main.columns = ['PARCEL_ID','COUNTY_ID','JURIS_NEW'] + list(plu_main)[3:]

# Convert all types to numeric to enable calculation
l = ['HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
     'ME_18','MT_18','MR_18','RB_18','RS_18','IH_18','IW_18','IL_18','SC_18','HO_18','OF_18','HM_18','HT_18','HS_18',
     'MAX_FAR_10','MAX_DUA_10','MAX_DUA_18','MAX_FAR_18','MAX_HEIGHT_10','MAX_HEIGHT_18']
for i in l:
    plu_main[i] = pd.to_numeric(plu_main[i], errors='coerce')

# fill 'NaN' with 0
plu_main.update(plu_main[l].fillna(0))

### 2.1 Allowed Development Type Statistics

In [71]:
cty = ctyMap.copy()
cty.set_index('ctyCode',inplace = True)

dfs = []
for i in ['HS','HT','HM','OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']:
    plu = i+'_10'
    boc = i+'_18'
    df = plu_main[['COUNTY_ID']+ [plu,boc]].groupby(['COUNTY_ID']).sum().reset_index()
    df.set_index('COUNTY_ID',inplace = True)
    dfs.append(df)

plu_boc_parcelCount_comp = pd.concat([cty] + dfs, axis=1,join='inner')
#display(plu_boc_parcelCount_comp)

,ctyName,HS_10,HS_18,HT_10,HT_18,HM_10,HM_18,OF_10,OF_18,HO_10,...,RS_10,RS_18,RB_10,RB_18,MR_10,MR_18,MT_10,MT_18,ME_10,ME_18
1.0,Alameda,308544.0,279178.0,163145.0,98626.0,117971.0,95443.0,24386.0,148745.0,8374.0,...,59353.0,99654.0,6899.0,22185.0,18346.0,59210.0,21166.0,74770.0,11707.0,76740.0
13.0,Contra Costa,273712.0,216027.0,115984.0,108716.0,36686.0,116364.0,17445.0,118874.0,6670.0,...,18504.0,101814.0,3962.0,80853.0,4453.0,51945.0,6882.0,88782.0,5733.0,79399.0
41.0,Marin,80289.0,66989.0,25842.0,22470.0,14291.0,24469.0,3092.0,27271.0,719.0,...,3134.0,24456.0,762.0,10129.0,1890.0,15825.0,2350.0,17792.0,2308.0,17569.0
55.0,Napa,39684.0,39667.0,12947.0,28283.0,10671.0,26506.0,2294.0,4894.0,7937.0,...,1565.0,7437.0,1209.0,4908.0,478.0,1279.0,1058.0,1782.0,1043.0,1885.0
75.0,San Francisco,11887.0,131900.0,143283.0,57022.0,71920.0,28380.0,5052.0,8094.0,60022.0,...,10785.0,16997.0,1753.0,514.0,13448.0,17178.0,14090.0,14684.0,3442.0,864.0
81.0,San Mateo,170416.0,159317.0,83479.0,53410.0,55032.0,58605.0,11667.0,61042.0,5515.0,...,9275.0,52444.0,5907.0,15612.0,7433.0,25627.0,4827.0,28770.0,4002.0,18998.0
85.0,Santa Clara,364912.0,365739.0,273546.0,263258.0,50022.0,88990.0,18802.0,79491.0,3930.0,...,193675.0,68620.0,2852.0,20914.0,21875.0,34648.0,28739.0,38814.0,8435.0,45618.0
95.0,Solano,109227.0,79414.0,57421.0,46444.0,16018.0,18394.0,5128.0,25797.0,2891.0,...,7321.0,49047.0,4310.0,9886.0,2092.0,14309.0,2918.0,10914.0,2834.0,12149.0
97.0,Sonoma,154446.0,150437.0,81427.0,98194.0,65463.0,88942.0,6777.0,62823.0,2120.0,...,53089.0,94026.0,3501.0,26434.0,5551.0,76764.0,6088.0,56806.0,5685.0,73892.0


In [72]:
devType = ['HM','MR','HS','RS','OF','IW','IL','IH','HT','HO','SC','RB','MT','ME']
for i in devType:
    plu = i+'_10'
    boc = i+'_18'
    plu_boc_parcelCount_comp[i+'_diff'] = plu_boc_parcelCount_comp[boc] - plu_boc_parcelCount_comp[plu]
    plu_boc_parcelCount_comp[i+'_diff_pct'] = plu_boc_parcelCount_comp[i+'_diff']/plu_boc_parcelCount_comp[plu]

plu_boc_type_diff = plu_boc_parcelCount_comp[[x+'_diff' for x in devType] + [x+'_diff_pct' for x in devType]]
#display(plu_boc_type_diff)

plu_boc_comp.to_csv('outputs/plu_boc_parcelCount_comp.csv')

,HM_diff,MR_diff,HS_diff,RS_diff,OF_diff,IW_diff,IL_diff,IH_diff,HT_diff,HO_diff,...,OF_diff_pct,IW_diff_pct,IL_diff_pct,IH_diff_pct,HT_diff_pct,HO_diff_pct,SC_diff_pct,RB_diff_pct,MT_diff_pct,ME_diff_pct
1.0,-22528.0,40864.0,-29366.0,40301.0,124359.0,33129.0,41255.0,24555.0,-64519.0,17252.0,...,5.099606,4.126682,3.148756,5.996337,-0.395470,2.060186,-0.398985,2.215683,2.532552,5.555053
13.0,79678.0,47492.0,-57685.0,83310.0,101429.0,57295.0,65334.0,3707.0,-7268.0,77363.0,...,5.814216,14.649706,13.972198,2.501350,-0.062664,11.598651,0.931258,19.407118,11.900610,12.849468
41.0,10178.0,13935.0,-13300.0,21322.0,24179.0,-335.0,605.0,-527.0,-3372.0,1230.0,...,7.819858,-0.271916,0.432143,-0.852751,-0.130485,1.710709,-0.284645,12.292651,6.571064,6.612218
55.0,15835.0,801.0,-17.0,5872.0,2600.0,-5537.0,-5727.0,112.0,15336.0,-3404.0,...,1.133391,-0.650799,-0.746675,1.696970,1.184522,-0.428877,0.402279,3.059553,0.684310,0.807287
75.0,-43540.0,3730.0,120013.0,6212.0,3042.0,56.0,181.0,441.0,-86261.0,-57923.0,...,0.602138,0.021366,0.045881,5.727273,-0.602032,-0.965029,-0.910978,-0.706788,0.042158,-0.748983
81.0,3573.0,18194.0,-11099.0,43169.0,49375.0,3233.0,7143.0,-2212.0,-30069.0,22931.0,...,4.232022,0.570899,1.027031,-0.619434,-0.360198,4.157933,1.930040,1.642966,4.960224,3.747126
85.0,38968.0,12773.0,827.0,-125055.0,60689.0,25139.0,55609.0,557.0,-10288.0,4930.0,...,3.227795,3.398540,4.605300,0.192003,-0.037610,1.254453,-0.776496,6.333100,0.350569,4.408180
95.0,2376.0,12217.0,-29813.0,41726.0,20669.0,6237.0,6107.0,5975.0,-10977.0,-62.0,...,4.030616,4.239973,1.910826,3.704278,-0.191167,-0.021446,2.394479,1.293735,2.740233,3.286874
97.0,23479.0,71213.0,-4009.0,40937.0,56046.0,18435.0,8365.0,15714.0,16767.0,25279.0,...,8.270031,5.311150,2.502243,17.557542,0.205915,11.924057,0.383083,6.550414,8.330815,11.997713


### 2.2 Caculate Build out capacity for each parcel

In [129]:
# Calculate capacity

def cap(df,nonResLs,reLs,zoning_yr,pba):
    """
    df: parcel data with PBA40 PLU ("_10") and BASIS BOC ("_18") attributes 
    nonResLs: a list of non-residential development types
    reLs: a list of residential development types, including HS, HT, HM
    zoning_yr: string, '_10' or '_18'
    pba: string, '_PBA40' or '_PBA50'
    """
    
    # a parcel is 'allowNonRes' is at least one of the non-residential development types is allowed
    df['allowNonRes'+zoning_yr] = df[nonResLs].sum(axis=1) > 0 

    # a parcel is 'allowRes' is at least one of the residential development types is allowed
    df['allowRes'+zoning_yr] = df[reLs].sum(axis=1) > 0
    
    # DUA calculations apply to parcels 'allowRes' and not marked as "nodev"
    df['units'+zoning_yr] = df['ACRES'] * df['MAX_DUA'+zoning_yr]
    
    # if DUA is null or zero, calculate assuming a HU is 1200 square feet and a floor is 11 feet high
    calUnits_idx = (df['allowRes'+zoning_yr] == True) & (df['MAX_DUA'+zoning_yr] == 0) & (df['NODEV'+pba] == 0)
    df.loc[calUnits_idx,'units'+zoning_yr] = df.loc[calUnits_idx,'ACRES'] * 43560 / 1200
    df.loc[calUnits_idx,'calc_unit'+zoning_yr] = 'Yuqi_calc'
    
    df.loc[(df['allowRes'+zoning_yr]== False) | (df['NODEV'+pba] == 1) ,'units'+zoning_yr] = 0

    # FAR calculations apply to parcels 'allowNonRes' and not marked as "nodev"
    df['sqft'+zoning_yr] = df['ACRES'] * df['MAX_FAR'+zoning_yr] * 43560
    
    # if FAR is null or zero, calculate assuming a floor is eleven feet hight to do the calc and land coverage 50%
    calSQFT_idx = (df['allowNonRes'+zoning_yr] == True) & (df['MAX_FAR'+zoning_yr] == 0) & (df['NODEV'+pba] == 0)
    df.loc[calSQFT_idx,'sqft'+zoning_yr] = df.loc[calSQFT_idx,'ACRES'] * 0.5 * (df.loc[calSQFT_idx,'MAX_HEIGHT'+zoning_yr] / 11)
    df.loc[calSQFT_idx,'calc_sqft'+zoning_yr] = 'Yuqi_calc'
    
    df.loc[(df['allowNonRes'+zoning_yr]== False) | (df['NODEV'+pba] == 1),'sqft'+zoning_yr] = 0
    
    return df[['PARCEL_ID','allowNonRes'+zoning_yr,'allowRes'+zoning_yr,'sqft'+zoning_yr,'units'+zoning_yr,'calc_unit'+zoning_yr,'calc_sqft'+zoning_yr]] 

In [130]:
nonRes = ['OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']
Res = ['HS', 'HT', 'HM','MR']

# Calculate PBA40 PLU capacity 
nonRes_10 = [x+'_10' for x in nonRes]
Res_10 = [x+'_10' for x in Res]
plu_main_10 = plu_main.copy()
for i in ['allowRes_10','allowNonRes_10','units_10','sqft_10','calc_unit_10',
          'calc_unit_10','calc_sqft_10']:
    plu_main_10[i] = np.nan
cap_10 = cap(plu_main_10,nonRes_10,Res_10,'_10','_PBA40')

# Calculate PBA50 BOC capacity 
nonRes_18 = [x+'_18' for x in nonRes]
Res_18 = [x+'_18' for x in Res]
plu_main_18 = plu_main.copy()
for i in ['allowRes_18','allowNonRes_18','units_18','sqft_18','calc_unit_18',
          'calc_unit_18','calc_sqft_18']:
    plu_main_18[i] = np.nan
cap_18 = cap(plu_main_18,nonRes_18,Res_18,'_18','_PBA50')

p10_capacity = plu_main.merge(cap_10, on = 'PARCEL_ID', how = 'left').merge(cap_18, on ='PARCEL_ID', how = 'left')
#display(p10_capacity.head())

,PARCEL_ID,COUNTY_ID,JURIS_NEW,ZONING_ID,GEOM_ID_S,ACRES,MAX_FAR_10,MAX_DUA_10,MAX_DUA_18,MAX_FAR_18,...,sqft_10,units_10,calc_unit_10,calc_sqft_10,allowNonRes_18,allowRes_18,sqft_18,units_18,calc_unit_18,calc_sqft_18
0,229116.0,1.0,livr,60126.0,10305106092872,3.360520,0.0,2.00000,0.00000,0.00,...,0.000000,6.721041,NaN,NaN,False,False,0.000000,0.000000,NaN,NaN
1,244166.0,1.0,livr,11903.0,11107351665227,1.294423,0.0,3.00000,14.00000,0.35,...,0.000000,3.883268,NaN,NaN,False,False,0.000000,0.000000,NaN,NaN
2,202378.0,1.0,hayw,11803.0,11030175960628,14.993605,0.0,8.70000,0.00000,0.00,...,0.000000,130.444362,NaN,NaN,True,False,0.000000,0.000000,NaN,Yuqi_calc
3,2004420.0,97.0,uson,12975.0,6381677629073,316.247146,0.0,0.00417,0.01666,0.00,...,0.000000,1.318751,NaN,NaN,True,True,503.120459,5.268677,NaN,Yuqi_calc
4,340332.0,1.0,frem,2511.0,314875459798,0.621275,0.0,23.00000,1.00000,0.01,...,1.468469,14.289334,NaN,Yuqi_calc,False,False,0.000000,0.000000,NaN,NaN


In [132]:
p10_capacity.to_csv('outputs/devCapacity_comp.csv', index = False)

### 2.3 Zoning build-out-capacity at jurisdiction and county levels

In [135]:
# BOC by jurisdiction function
def boc_j(df):
    boc_j = df.groupby(['JURIS_NEW'])['ACRES','units_10','units_18','sqft_10','sqft_18'].sum()

    boc_j['unit_diff'] = boc_j['units_18'] - boc_j['units_10']
    boc_j['sqft_diff'] = boc_j['sqft_18'] - boc_j['sqft_10']
    boc_j['unit_diff_pct'] = boc_j['unit_diff'] / boc_j['units_10']
    boc_j['sqft_diff_pct'] = boc_j['sqft_diff'] / boc_j['sqft_10']

    for i in ['units_10','units_18','unit_diff','sqft_10','sqft_18','sqft_diff']:
        boc_j[i] = boc_j[i].apply(lambda x: f'{int(x):,}')
    #display(boc_j)
    return boc_j

# BOC by county function
def boc_c(df):
    boc_cty = df.groupby(['COUNTY_ID'])['ACRES','units_10','units_18','sqft_10','sqft_18'].sum()
    boc_cty['unit_diff'] = boc_cty['units_18'] - boc_cty['units_10']
    boc_cty['sqft_diff'] = boc_cty['sqft_18'] - boc_cty['sqft_10']
    boc_cty['unit_diff_pct'] = boc_cty['unit_diff'] / boc_cty['units_10']
    boc_cty['sqft_diff_pct'] = boc_cty['sqft_diff'] / boc_cty['sqft_10']

    for i in ['units_10','units_18','unit_diff','sqft_10','sqft_18','sqft_diff']:
        boc_cty[i] = boc_cty[i].apply(lambda x: f'{int(x):,}')

    boc_cty = boc_cty.reset_index()
    boc_cty = boc_cty.loc[boc_cty['COUNTY_ID'] > 0]

    boc_c = boc_cty.merge(ctyMap, left_on = 'COUNTY_ID', right_on = 'ctyCode', how = 'left')
    #display(boc_c)
    return boc_c

In [136]:
# all parcels statistics
all_boc_j = boc_j(p10_capacity)
all_boc_j.to_csv('outputs/all_boc_jurisdiction.csv')

all_boc_c = boc_c(p10_capacity)
all_boc_c.to_csv('outputs/all_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [139]:
# vacant parcel statistics

p_vac = p10_capacity.loc[p10_capacity.VACANT == 'vacant']

vac_boc_j = boc_j(p_vac)
vac_boc_j.to_csv('outputs/vac_boc_jurisdiction.csv')

vac_boc_c = boc_c(p_vac)
vac_boc_c.to_csv('outputs/vac_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [140]:
# low ILR parcel statistics (threadhold 0.2)
p10_capacity.ILR = pd.to_numeric(p10_capacity.ILR, errors='coerce')
p_low_ILR = p10_capacity.loc[p10_capacity.ILR < 0.2]

low_ILR_boc_j = boc_j(p_low_ILR)
low_ILR_boc_j.to_csv('outputs/low_ILR_boc_jurisdiction.csv')

low_ILR_boc_c = boc_c(p_low_ILR)
low_ILR_boc_c.to_csv('outputs/low_ILR_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [141]:
# Old building parcel statistics (1930-1980)
p10_capacity.year_built = pd.to_numeric(p10_capacity.YEAR_BUILT, errors='coerce')
p_old = p10_capacity.loc[(p10_capacity.YEAR_BUILT < 1980) & (p10_capacity.YEAR_BUILT >= 1930)]

old_boc_j = boc_j(p_old)
old_boc_j.to_csv('outputs/old_boc_jurisdiction.csv')

old_boc_c = boc_c(p_old)
old_boc_c.to_csv('outputs/old_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
